# 유튜브 인기 동영상 데이터

In [1]:
import pandas as pd 
import numpy as np

import warnings
warnings.filterwarnings('ignore')

데이터 로드

In [2]:
dataurl = "https://raw.githubusercontent.com/Datamanim/datarepo/main/youtube/youtube.csv"
df = pd.read_csv(dataurl, index_col=0)

print(df.shape)
df.head(2)

(60399, 9)


,title,channelTitle,categoryId,view_count,likes,dislikes,comment_count,channelId,trending_date2
0,[신병] 물자창고,장삐쭈,23,1893473,38249,730,8595,UChbE5OZQ6dRHECsX0tEPEZQ,2021-01-01
1,RAIN(비) - 나로 바꾸자 Switch to me (duet with JYP) MV,RAIN's Official Channel,10,2600864,0,0,20129,UCxXgIeE5hxWxHG6dz9Scg2w,2021-01-01


1. 인기동영상 제작횟수가 많은 채널 상위 10개명을 출력하라 (날짜기준, 중복포함)

In [3]:
id = df['channelId'].value_counts().head(10).index # 제작횟수가 많은 channelid
ans = list(df[df['channelId'].isin(id)]['channelTitle'].unique()) # 제작횟수가 많은 channelid의 channeltitle명
ans

['장삐쭈',
 '총몇명',
 '파뿌리',
 '짤툰',
 '런닝맨 - 스브스 공식 채널',
 '엠뚜루마뚜루 : MBC 공식 종합 채널',
 'SPOTV',
 '채널 십오야',
 '이과장',
 'BANGTANTV']

2. 논란으로 인기동영상이 된 케이스를 확인하고 싶다. dislikes수가 like 수보다 높은 동영상을 제작한 채널을 모두 출력하라

In [4]:
ans = list(df[df['dislikes'] > df['likes']]['channelTitle'].unique())
ans

['핫도그TV',
 'ASMR 애정TV',
 '하얀트리HayanTree',
 '양팡 YangPang',
 '철구형 (CHULTUBE)',
 '왜냐맨하우스',
 '(MUTUBE)와꾸대장봉준',
 '오메킴TV',
 '육지담',
 'MapleStory_KR',
 'ROAD FIGHTING CHAMPIONSHIP',
 '사나이 김기훈',
 '나혼자산다 STUDIO',
 'Gen.G esports']

3. 채널명을 바꾼 케이스가 있는지 확인하고 싶다. channelId의 경우 고유값이므로 이를 통해 채널명을 한번이라도 바꾼 채널의 갯수를 구하여라

In [5]:
change = df[['channelId', 'channelTitle']].drop_duplicates().channelId.value_counts()
ans = len(change[change > 1])
ans

71

4. 일요일에  인기있었던 영상들중 가장많은 영상 종류(categoryId)는 무엇인가?

In [6]:
df['trending_date2'] = pd.to_datetime(df['trending_date2'])

ans = df[df['trending_date2'].dt.day_name() == 'Sunday']['categoryId'].value_counts().index[0]
ans

24

5. 각 요일별 인기 영상들의 categoryId는 각각 몇개 씩인지 하나의 데이터 프레임으로 표현하라

In [7]:
ans = df.groupby([df['trending_date2'].dt.day_name(), 'categoryId'], as_index=False).size()
ans = ans.pivot(index='categoryId', columns='trending_date2', values='size')
ans

trending_date2,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
categoryId,,,,,,,
1,243,263,255,274,246,257,234
2,120,105,119,99,128,119,129
10,833,837,776,830,890,894,917
15,187,215,198,217,207,208,207
17,633,668,592,636,682,708,706
19,90,92,87,91,92,89,85
20,283,298,296,289,282,285,291
22,1288,1373,1289,1337,1341,1375,1333
23,568,594,570,556,560,569,566


6. 댓글의 수로 (comment_count) 영상 반응에 대한 판단을 할 수 있다. viewcount대비 댓글수가 가장 높은 영상을 확인하라 (view_count값이 0인 경우는 제외한다)

In [8]:
target = df[df['view_count'] != 0].copy()
target['ratio'] = target['comment_count'] / df['view_count']

ans = target.sort_values(by='ratio', ascending=False).iloc[0].title
ans

'60분 동안 댓글이 달리지 않으면, 영상이 삭제됩니다. (챌린지)'

7. 댓글의 수로 (comment_count) 영상 반응에 대한 판단을 할 수 있다.viewcount대비 댓글수가 가장 낮은 영상을 확인하라 (view_counts, ratio값이 0인경우는 제외한다.)

In [9]:
target = df[df['view_count'] != 0]
target['ratio'] = df['comment_count'] / df['view_count']

ans = target[target['ratio'] != 0].sort_values(by='ratio').iloc[0].title
ans

'Join the BTS #PermissiontoDance Challenge only on YouTube #Shorts'

8. like 대비 dislike의 수가 가장 적은 영상은 무엇인가? (like, dislike 값이 0인경우는 제외한다)

In [10]:
target = df[(df['likes'] != 0) & (df['dislikes'] != 0)].copy()
target['ratio'] = df['dislikes'] / df['likes']

ans = target.sort_values(by='ratio').iloc[0].title
ans

'[줌터뷰] *최초공개* 사부작즈🐰🐶의 비공식 이름은 아이라인즈? 꿀조합 티키타카 가득한 NCT 127 도영&정우의 줌터뷰'

9. 가장 많은 트렌드 영상을 제작한 채널의 이름은 무엇인가? (날짜기준, 중복포함)

In [11]:
ans = df[df['channelId'] == df['channelId'].value_counts().index[0]]['channelTitle'].unique()[0]
ans

'짤툰'

10. 20회(20일)이상 인기동영상 리스트에 포함된 동영상의 숫자는?

In [12]:
ans = (df[['title', 'channelTitle']].value_counts() >= 20).sum()
ans

40

# 유튜브 공범컨텐츠 동영상 데이터

데이터 로드

In [13]:
channel = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/youtube/channelInfo.csv')
video = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/youtube/videoInfo.csv')

In [14]:
print(channel.shape)
channel.head(2)

(1395, 6)


,channelid,subcnt,viewcnt,videocnt,ct,channelname
0,UCkQCwnkQfgSuPTTnw_Y7v7w,1310000,410238653,736,2021-09-30 03:01:03,꽈뚜룹
1,UCkQCwnkQfgSuPTTnw_Y7v7w,1310000,412531322,736,2021-09-30 09:01:03,꽈뚜룹


In [15]:
print(video.shape)
video.head(2)

(15201, 8)


,videopk,viewcnt,likecnt,dislikecnt,favoritecnt,cmcnt,ct,videoname
0,c5JQp6xafqc,1667010,30474,706,0,6587,2021-10-10 15:20:03,공범 EP1
1,c5JQp6xafqc,1669089,30495,707,0,6589,2021-10-10 15:30:03,공범 EP1


11. 각 데이터의 'ct'컬럼을 시간으로 인식할수 있게 datatype을 변경하고 video 데이터의 videoname의 각 value 마다 몇개의 데이터씩 가지고 있는지 확인하라

In [16]:
channel['ct'] = pd.to_datetime(channel['ct'])
video['ct'] = pd.to_datetime(video['ct'])

ans = video['videoname'].value_counts()
ans

videoname
 공범 EP1    3492
 공범 EP2    3204
 공범 EP3    2568
 공범 EP4    2280
 공범 EP5    1562
 공범 EP6    1274
 공범 EP7     555
 공범 EP8     266
Name: count, dtype: int64

12. 수집된 각 video의 가장 최신화 된 날짜의 viewcount값을 출력하라

In [17]:
ans = video.sort_values(by=['videoname', 'ct']).drop_duplicates(subset='videoname', keep='last')[['videoname', 'ct', 'viewcnt']].reset_index()
ans

,index,videoname,ct,viewcnt
0,3491,공범 EP1,2021-11-01 15:30:03,3180532
1,6695,공범 EP2,2021-11-01 15:30:03,2199328
2,9263,공범 EP3,2021-11-01 15:30:03,1671294
3,11543,공범 EP4,2021-11-01 15:30:03,1818493
4,13105,공범 EP5,2021-11-01 15:30:04,1503435
5,14934,공범 EP6,2021-11-01 15:30:04,1750222
6,13660,공범 EP7,2021-11-01 15:30:05,1630200
7,15200,공범 EP8,2021-11-01 15:30:05,1289088


13. Channel 데이터 중 2021-10-03일 이후 각 채널의 처음 기록 됐던 구독자 수(subcnt)를 출력하라

In [18]:
ans = channel[channel['ct'] >= '2021-10-03'].sort_values(by=['channelname', 'ct']).drop_duplicates(subset='channelname', keep='first')
ans = ans[['channelname', 'subcnt']].reset_index()
ans

,index,channelname,subcnt
0,1276,Balming Tiger,54300
1,1155,곽토리 kwak tori,471000
2,901,김농밀의 농밀한 삶,7520
3,12,꽈뚜룹,1330000
4,393,논리왕 전기,922000
5,1028,릴펄 Lilpearl,10100
6,139,야전삽짱재,257000
7,520,와글와글 WagleWagle,55000
8,647,조나단,215000
9,266,츄정ChuJeong,322000


14. 각 채널의 2021-10-03 03:00:00 ~ 2021-11-01 15:00:00 까지 구독자수 (subcnt) 의 증가량을 구하여라

In [19]:
start = channel[channel['ct'].dt.strftime('%Y-%m-%d %H') == '2021-10-03 03'].copy()
end = channel[channel['ct'].dt.strftime('%Y-%m-%d %H') == '2021-11-01 15'].copy()

start_df = start[['channelname', 'subcnt']].reset_index(drop=True)
end_df = end[['channelname', 'subcnt']].reset_index(drop=True)

start_df.columns = ['channelname', 'start_subcnt']
end_df.columns = ['channelname', 'end_subcnt']

ans = pd.merge(start_df, end_df)
ans['del'] = ans['end_subcnt'] - ans['start_subcnt']
ans = ans[['channelname', 'del']]
ans

,channelname,del
0,꽈뚜룹,70000
1,야전삽짱재,11000
2,츄정ChuJeong,1000
3,논리왕 전기,-11000
4,와글와글 WagleWagle,0
5,조나단,12000
6,형사!탐정되다,10300
7,김농밀의 농밀한 삶,1540
8,릴펄 Lilpearl,11000
9,곽토리 kwak tori,-2000


15. 각 비디오는 10분 간격으로 구독자수, 좋아요, 싫어요수, 댓글수가 수집된것으로 알려졌다. 공범 EP1의 비디오정보 데이터 중 수집간격이 5분 이하, 20분이상인 데이터 구간(해당 시점 전,후) 의 시각을 모두 출력하라

In [20]:
import datetime

# 공범 EP1
ep_1 = video[video['videoname'].str.contains('1')].sort_values(by='ct').reset_index(drop=True)

ep_1 = ep_1[(ep_1['ct'].diff(1) <= datetime.timedelta(minutes=5)) | 
            (ep_1['ct'].diff(1) >= datetime.timedelta(minutes=20))].copy()

ans = ep_1.copy()
ans

,videopk,viewcnt,likecnt,dislikecnt,favoritecnt,cmcnt,ct,videoname
721,c5JQp6xafqc,2228250,34559,849,0,7191,2021-10-13 09:41:37,공범 EP1
722,c5JQp6xafqc,2228250,34559,849,0,7191,2021-10-13 09:41:37,공범 EP1
1636,c5JQp6xafqc,2707933,37901,988,0,7410,2021-10-19 18:20:03,공범 EP1


16. 각 에피소드의 시작날짜(년-월-일)를 에피소드 이름과 묶어 데이터 프레임으로 만들고 출력하라

In [21]:
ans = video.sort_values(by=['videoname', 'ct']).drop_duplicates(subset='videoname', keep='first')[['videoname', 'ct']].reset_index(drop=True)
ans['ct'] = ans['ct'].dt.date
ans

,videoname,ct
0,공범 EP1,2021-10-07
1,공범 EP2,2021-10-09
2,공범 EP3,2021-10-14
3,공범 EP4,2021-10-16
4,공범 EP5,2021-10-21
5,공범 EP6,2021-10-23
6,공범 EP7,2021-10-28
7,공범 EP8,2021-10-30


17. "공범" 컨텐츠의 경우 19:00시에 공개 되는것으로 알려져있다. 공개된 날의 21시의 viewcnt, ct, videoname 으로 구성된 데이터 프레임을 viewcnt를 내림차순으로 정렬하여 출력하라

In [22]:
ans = video[video['ct'].dt.hour == 21].sort_values(by=['videoname', 'ct']).drop_duplicates(subset='videoname', keep='first').reset_index(drop=True)
ans = ans[['videoname', 'ct', 'viewcnt']].sort_values(by='viewcnt', ascending=False)
ans

,videoname,ct,viewcnt
7,공범 EP8,2021-10-30 21:00:08,264029
6,공범 EP7,2021-10-28 21:00:04,252032
3,공범 EP4,2021-10-16 21:00:04,217674
5,공범 EP6,2021-10-23 21:00:06,213899
4,공범 EP5,2021-10-21 21:00:04,201179
1,공범 EP2,2021-10-09 21:00:03,148144
2,공범 EP3,2021-10-14 21:00:04,147183
0,공범 EP1,2021-10-07 21:00:03,117340


18. video 정보의 가장 최근 데이터들에서 각 에피소드의 싫어요/좋아요 비율을 ratio 컬럼으로 만들고 videoname, ratio로 구성된 데이터 프레임을 ratio를 오름차순으로 정렬하라

In [23]:
ans = video.sort_values(by='ct', ascending=False).drop_duplicates(subset='videoname', keep='first').reset_index(drop=True)
ans['ratio'] = ans['dislikecnt'] / ans['likecnt']
ans = ans[['videoname', 'ratio']].sort_values(by='ratio')
ans

,videoname,ratio
1,공범 EP7,0.013636
2,공범 EP6,0.015881
6,공범 EP4,0.017971
3,공범 EP5,0.021128
7,공범 EP3,0.022360
4,공범 EP1,0.027118
5,공범 EP2,0.029844
0,공범 EP8,0.141357


19. 2021-11-01 00:00:00 ~ 15:00:00까지 각 에피소드별 viewcnt의 증가량을 데이터 프레임으로 만드시오

In [24]:
start = pd.to_datetime('2021-11-01 00:00:00')
end = pd.to_datetime('2021-11-01 15:00:00')

ans = video[(video['ct'] >= start) & (video['ct'] <= end)].reset_index(drop=True)

# 최댓값과 최솟값 차이 함수
def check(x) : 
    return max(x) - min(x)

ans = ans[['videoname', 'viewcnt']].groupby('videoname').agg(check)
ans

,viewcnt
videoname,
공범 EP1,13298
공범 EP2,10300
공범 EP3,9927
공범 EP4,9824
공범 EP5,10824
공범 EP6,14141
공범 EP7,26949
공범 EP8,89147


20. video 데이터 중에서 중복되는 데이터가 존재한다. 중복되는 각 데이터의 시간대와  videoname 을 구하여라

In [25]:
ans = video[video.duplicated()][['videoname', 'ct']].copy()
ans

,videoname,ct
722,공범 EP1,2021-10-13 09:41:37
3927,공범 EP2,2021-10-13 09:41:37


# 월드컵 출전 선수 골기록 데이터

데이터 로드

In [26]:
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/worldcup/worldcupgoals.csv')

print(df.shape)
df.head()

(1295, 4)


,Player,Goals,Years,Country
0,Miroslav Klose,16,2002-2006-2010-2014,Germany
1,Ronaldo,15,1998-2002-2006,Brazil
2,Gerd Muller,14,1970-1974,Germany
3,Just Fontaine,13,1958,France
4,Pele,12,1958-1962-1966-1970,Brazil


21. 주어진 전체 기간의 각 나라별 골득점수 상위 5개 국가와 그 득점수를 데이터프레임형태로 출력하라

In [31]:
ans = df.groupby('Country').sum().sort_values(by='Goals', ascending=False).head(5)
ans

,Player,Goals,Years
Country,,,
Brazil,RonaldoPeleAdemirJairzinhoVavaLeonidasRivaldoC...,228,1998-2002-20061958-1962-1966-197019501966-1970...
Germany,Miroslav KloseGerd MullerJurgen KlinsmannThoma...,226,2002-2006-2010-20141970-19741990-1994-19982010...
Argentina,Gabriel BatistutaDiego MaradonaGuillermo Stabi...,135,1994-1998-20021982-1986-1990-199419301974-1978...
Italy,Roberto BaggioPaolo RossiChristian VieriSalvat...,123,1990-1994-19981978-19821998-2002199019381982-1...
France,Just FontaineThierry HenryMichel PlatiniZinedi...,115,19581998-2002-2006-20101978-1982-19861998-2002...


22. 주어진 전체기간동안 골득점을 한 선수가 가장 많은 나라 상위 5개 국가와 그 선수 숫자를 데이터 프레임 형식으로 출력하라

In [40]:
ans = df.groupby('Country')['Player'].size().sort_values(ascending=False).head(5)
ans

Country
Brazil       81
Germany      78
Italy        60
Argentina    59
France       58
Name: Player, dtype: int64

23. Years 컬럼은 년도 -년도 형식으로 구성되어있고, 각 년도는 4자리 숫자이다. 년도 표기가 4자리 숫자로 안된 케이스가 존재한다. 해당 건은 몇건인지 출력하라

In [47]:
df['Years_list'] = df['Years'].str.split('-')

# 연도 4자리 표기 확인 함수
def check(x) :
    for i in x :
        if len(str(i)) != 4 :
            return 1
    return 0 

df['check'] = df['Years_list'].apply(check)

ans = df['check'].sum()
ans

45

In [48]:
# 확인
df[df['check'] == 1].head()

,Player,Goals,Years,Country,Years_list,check
71,Roger Milla,5,0,Cameroon,[0],1
190,Jesper Olsen,3,0,Denmark,[0],1
210,Laszlo Kiss,3,0,Hungary,[0],1
404,Ricardo Pelaez,2,0,Mexico,[0],1
505,Djamel Zidane,1,0,Algeria,[0],1


24. 문제 23번에서 발생한 예외 케이스를 제외한 데이터프레임을 df2라고 정의하고 데이터의 행의 숫자를 출력하라 (아래 문제부터는 df2로 풀이하겠습니다)

In [52]:
df2 = df[df['check'] != 1].reset_index(drop=True)
ans = df2.shape[0]
ans

1250

25. 월드컵 출전횟수를 나타내는 'LenCup' 컬럼을 추가하고 4회 출전한 선수의 숫자를 구하여라

In [59]:
df2['LenCup'] = df2['Years_list'].str.len()
ans = df2['LenCup'].value_counts()[4]
ans

16

26. Yugoslavia 국가의 월드컵 출전횟수가 2회인 선수들의 숫자를 구하여라

In [63]:
ans = len(df2[(df2['Country'] == 'Yugoslavia') & (df2['LenCup'] == 2)])
ans

7

27. 2002년도에 출전한 전체 선수는 몇명인가?

In [72]:
ans = len(df2[df2['Years'].str.contains('2002')])
ans

156

28. 이름에 'carlos' 단어가 들어가는 선수의 숫자는 몇 명인가? (대, 소문자 구분 x)

In [77]:
ans = len(df2[df2['Player'].str.lower().str.contains('carlos')])
ans

13

29. 월드컵 출전 횟수가 1회뿐인 선수들 중에서 가장 많은 득점을 올렸던 선수는 누구인가?

In [83]:
ans = df2[df2['LenCup'] == 1].sort_values(by='Goals', ascending=False).iloc[0].Player
ans

'Just Fontaine'

30. 월드컵 출전횟수가 1회 뿐인 선수들이 가장 많은 국가는 어디인가?

In [87]:
ans = df2[df2['LenCup'] == 1]['Country'].value_counts().index[0]
ans

'Brazil'